In [22]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import *  # noqa
from threading import Thread
import time
import nest_asyncio
nest_asyncio.apply()

import re
from datetime import datetime, timedelta

In [23]:
class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.headlines = []  # List to store news headlines
        self.price_data = []
        self.conId = None  # Store the retrieved conId
        self.news_providers = []  # Store available news providers

    def contractDetails(self, reqId, contractDetails):
        self.conId = contractDetails.contract.conId

    def newsProviders(self, newsProviders):
        self.news_providers = [provider.code for provider in newsProviders]

    def historicalData(self, reqId: int, bar):
        self.price_data.append({
            "time": bar.date,
            "open": bar.open,
            "high": bar.high,
            "low": bar.low,
            "close": bar.close,
            "volume": bar.volume
        })

    def historicalNews(self, reqId: int, time: str, providerCode: str, articleId: str, headline: str):
        # Save each headline to the list
        self.headlines.append({
            "time": time,
            "providerCode": providerCode,
            "articleId": articleId,
            "headline": headline
    })

def run_loop(api):
    api.run()

def clean_news_headlines(news_headlines):
    ''' Helper '''
    
    query = []
    for news in news_headlines:
        cleaned = re.sub(r'\{.*?\}', '', news['headline']) + '- ' + news['time']
        query.append(cleaned)

    return query

In [24]:
# Initialize the IBApi object
app = IBApi()

# Connect to TWS or IB Gateway
app.connect("127.0.0.1", 7497, clientId=1)

# Start the API thread
api_thread = Thread(target=run_loop, args=(app,), daemon=True)
api_thread.start()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [25]:
def get_Data(contract, endDate, historicalDataParams, historicalNewsParam):

    historical_data_format = endDate.strftime("%Y%m%d %H:%M:%S") + " US/Eastern"
    historical_news_format = endDate.strftime("%Y-%m-%d %H:%M:%S") + ".0"

    app.reqContractDetails(1, contract)
    time.sleep(2)  # Wait for the conId to be retrieved
    if app.conId is None:
        raise Exception("Failed to retrieve conId for the contract")

    app.reqNewsProviders()
    time.sleep(2)  # Wait for news providers to be retrieved
    if not app.news_providers:
        raise Exception("Failed to retrieve news providers")

    news_providers = "+".join(app.news_providers)
    
    # app.reqHistoricalData(
    #     reqId=1,
    #     contract=historicalDataParams['contract'],
    #     endDateTime=historical_data_format,  # End of the day in UTC
    #     durationStr="1 D",                    # 1 day duration
    #     barSizeSetting="2 hours",             # 2-hour bars
    #     whatToShow="MIDPOINT",                # Midpoint prices
    #     useRTH=1,                             # Regular Trading Hours only
    #     formatDate=1,
    #     keepUpToDate=False,
    #     chartOptions=None                     # No special chart options
    # )

    app.reqHistoricalData(
        reqId=1,
        contract=historicalDataParams['contract'],
        endDateTime=historical_data_format,  # End of the day in UTC
        durationStr=historicalDataParams['historicalDataDuration'],
        barSizeSetting=historicalDataParams['frequency'],             # 2-hour bars
        whatToShow="MIDPOINT",                # Midpoint prices
        useRTH=1,                             # Regular Trading Hours only
        formatDate=1,
        keepUpToDate=False,
        chartOptions=None                     # No special chart options
    )
    
    # Request historical news for the contract
    time.sleep(2)  # Wait for connection #Use the startdate, I havent figured how the date range works but we can simply filter out the news later on. :)
    app.reqHistoricalNews(1, app.conId, news_providers, historical_news_format, "", historicalNewsParam['numberOfHeadlines'], None)  # Fetch the latest 10 news items

    
    # Wait for news to fetch
    time.sleep(5)
    
    # Save headlines to a variable
    news_headlines = clean_news_headlines(app.headlines)
    prices = app.price_data

    return prices, news_headlines

In [26]:
contract = Contract()
contract.symbol = "AMD"
contract.secType = "STK"
contract.exchange = "SMART"
contract.currency = "USD"

endDate = datetime(2024, 12, 11, 21, 44, 0)

In [27]:
historicalDataParams = {
    'contract': contract,
    'historicalDataDuration': '1 W',  # One week
    'frequency': '2 hours',
}

historicalNewsParam = {
    'numberOfHeadlines' : 50
}

In [28]:
a, b = get_Data(contract, endDate, historicalDataParams, historicalNewsParam)

In [32]:
b

["Nvidia Stock Rises. 3 Reasons to Look Past China's Antitrust Probe. -- Barrons.com- 2024-12-11 18:33:00.0",
 'AMD, Broadcom, Marvell Are Trailing Nvidia. This AI Chip Stock Can Take Second Spot. -- Barrons.com- 2024-12-11 15:20:00.0',
 "Nvidia Stock Rises. 3 Reasons to Look Past China's Antitrust Probe. -- Barrons.com- 2024-12-11 15:14:00.0",
 "AMD, Broadcom, Marvell Are in Nvidia's Wake. This AI Chip Stock Can Take Second Spot. -- Barrons.com- 2024-12-11 13:23:00.0",
 "Nvidia Stock Rises. 3 Reasons to Look Past China's Antitrust Probe. -- Barrons.com- 2024-12-11 11:44:00.0",
 'Nvidia Stock Gains. What TSMC Sales, China Probe Mean for Chip Maker. -- Barrons.com- 2024-12-10 15:22:00.0',
 "Press Release: AMD Introduces Versal RF Series Adaptive SoCs Offering the Industry's Highest Compute in a Single-Chip Device with Integrated Direct RF-Sampling Converters- 2024-12-10 14:00:00.0",
 'Nvidia Stock Slips. TSMC Revenue Boost Fails to Offset Shadow of China Probe. -- Barrons.com- 2024-12-1

In [21]:
app.disconnect()